In [6]:
# Imports
import pandas as pd
import autograd.numpy as np  # Thinly-wrapped numpy
from autograd import grad 
from autograd import elementwise_grad
from matplotlib import pyplot as plt
from numpy.random import multivariate_normal as N
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix

In [2]:
# Read in the data
# Each row is formatted as [User ID] [Joke ID] [Rating]
ratingsFile =  "ratings.dat"
allData = pd.read_csv(ratingsFile, sep=" ", header=None)
allData = allData.astype(int)

In [3]:
allData.columns = ['UserID', 'JokeID', 'Rating']

In [204]:
def splitData(R, trainPercent=0.95):
    # R is an NxM matrix where r_ij is the rating assigned by user i to jokes.
    # Here, we split the data into matrices of size ratings x 2, where the two columns are JokeID and Rating
    N,M = R.shape
    totalRatings = np.count_nonzero(R)
    testMat = np.zeros((totalRatings,2)) 
    trainMat = np.zeros((totalRatings, 2))
    trainN, testN = 0,0
    print N,M
    for j in range(M):
        tratings = float(np.count_nonzero(R[:,j]))
        inTraining = 0
        for i in range(N):
            if R[i,j] > 0:
                if inTraining <= (tratings * trainPercent):
                    inTraining += 1
                    trainMat[trainN, :] = np.array([j, R[i,j]])
                    trainN += 1
                else:
                    testMat[testN, :] = np.array([j, R[i,j]])
                    testN += 1
    # Note that the results are zero indexed! We've mofidied the joke ids so now they are 0-indexed!!!!!
    return trainMat, testMat

In [32]:
# by user i to joke j
R = np.zeros((max(allData.UserID), max(allData.JokeID)))
for row in allData.iterrows():
    r = row[1]
    R[r['UserID']-1, r['JokeID']-1] = r['Rating']
# np.savetxt('all_ratings.txt', R)

In [28]:
R = np.loadtxt('all_ratings.txt')

In [95]:
train, test = splitData(R)

63978 150


In [105]:
# drop all 0s from the data
def dropZeroRows(a):
    mask = np.all(np.isnan(a) | np.equal(a, 0), axis=1)
    return a[~mask]

In [213]:
trainE, testE = dropZeroRows(train), dropZeroRows(test)
trainE, testE = trainE.astype(int), testE.astype(int)

In [107]:
np.savetxt('95trainE.txt', trainE)
np.savetxt('5testE.txt', testE)

In [109]:
len(trainE)+len(testE), len(allData)

(1761439, 1761439)

In [293]:
from autograd.scipy.stats import norm
import sys
def log_lik(weights, x_i, br1, br2):
    # br1 and br2 is a vector of respecive rankings.
    # We add the np.sum to vectorize the below into batches. Note that x_i can now be a matrix
    # print weights.shape, x_i.shape, br1.shape, br2.shape
    cdf = lambda y: norm.cdf( (y - np.dot(weights[:-1], np.transpose(x_i)))/weights[-1])
    return np.log(cdf(br2) - cdf(br1))

def log_prior(w):
    return np.sum(norm.logpdf(w))

def scaled_posterior(weights, x_i, br1, br2):
    # Same parameters as log_lik.
    # Note that we add up all the results to be able to batch process the final posterior probability
    return np.sum(log_lik(weights, x_i, br1, br2) + log_prior(weights[:-1]))

In [294]:
posteriorGradient = grad(scaled_posterior, argnum=0)

In [88]:
# load the feature space matrix
# Solution for sigma^2 is 16, or roughly 16
X = np.loadtxt('X.txt')

In [90]:
X.shape

(150, 151)

In [295]:
def adam(grad, x, num_iters=100,
         step_size=0.001, b1=0.9, b2=0.999, eps=10**-8):
    """
    Adam as described in http://arxiv.org/pdf/1412.6980.pdf.
    It's basically RMSprop with momentum and some correction terms.
    Code directly copied from autograd package and modified slightly,
    as it now maximizes rather than minimizes. 
    """
    m = np.zeros(len(x))
    v = np.zeros(len(x))
    for i in range(num_iters):
        g = grad(x)
        m = (1 - b1) * g      + b1 * m  # First  moment estimate.
        v = (1 - b2) * (g**2) + b2 * v  # Second moment estimate.
        mhat = m / (1 - b1**(i + 1))    # Bias correction.
        vhat = v / (1 - b2**(i + 1))
        x += step_size*mhat/(np.sqrt(vhat) + eps)
    return x

In [315]:
b = [-sys.maxint/10, -4, -2, 2, 4, sys.maxint/10]
def ratingToLowerBound(r):
    # Takes in a rating and returns br1, the lower bound in our intervals
    # Note that ratings are 1 indexed
    return b[r-1]
    
def ratingToUpperBound(r):
    return b[r]

lowVect = np.vectorize(ratingToLowerBound)
highVect= np.vectorize(ratingToUpperBound)

def stochasticGradientDescent(T, data, batchSize):
    (n, m) = data.shape
    #print (n,m)
    w0 = N(mean=np.zeros(X.shape[1] + 1),cov=np.identity(X.shape[1] + 1))
    #print w0.shape
    # Precomput br1 and br2 values
    br1all = lowVect(data[:, 1])
    br2all = highVect(data[:, 1])
    for epoch in range(T):
        # Iterate over the data size with epochs
        for i in xrange(0, len(data), batchSize):
            batchJokeIDs = data[i:min(len(data),i+batchSize), 0]
            ratings = data[i:min(len(data),i+batchSize), 1]
            xi = X[batchJokeIDs, :]
            br1 = br1all[i:min(len(data),i+batchSize)]
            br2 = br2all[i:min(len(data),i+batchSize)]
            w0 = adam(lambda w: posteriorGradient(w0, xi, br1, br2), w0)
            if i % batchSize == 0:
                print "A total of {} points have been completed.".format(i)
        print "Finished epoch {}".format(epoch)
            
    return w0

In [316]:
len(trainE)

1673440

In [ ]:
w = stochasticGradientDescent(10, trainE, 100000)

A total of 0 points have been completed.
A total of 100000 points have been completed.

In [319]:
wFinal = np.array([  4.22455732e-02,   4.22473088e-02,   5.37630673e-06,
         4.22455732e-02,   5.37596623e-06,   4.22473088e-02,
         4.22455732e-02,   4.22473088e-02,   4.22473221e-02,
         4.22473222e-02,   4.22455732e-02,  -2.54292998e-06,
         4.22455696e-02,   4.22455732e-02,   5.37532906e-06,
         5.37634719e-06,  -5.21942825e-06,  -5.21937071e-06,
        -5.21942778e-06,   4.22455732e-02,   5.37634719e-06,
         5.37634675e-06,   5.37634719e-06,   4.22455696e-02,
         4.22473086e-02,   5.21942780e-06,   5.37533456e-06,
         4.22455732e-02,   5.37533405e-06,  -2.78179336e-06,
         4.22455732e-02,  -5.21942878e-06,   5.21229106e-06,
        -5.21943212e-06,  -2.78179336e-06,   4.22473222e-02,
         4.22455732e-02,   4.22455696e-02,   5.21229121e-06,
        -2.78179361e-06,   4.22455732e-02,  -2.40925210e-06,
        -5.21942778e-06,   5.37630675e-06,  -5.21942825e-06,
         4.22455696e-02,   4.22455696e-02,  -2.78179294e-06,
         4.22455696e-02,  -5.21942748e-06,   5.21229106e-06,
         5.37533445e-06,  -2.78179294e-06,  -5.21942748e-06,
         4.22455696e-02,   5.21229209e-06,  -5.21942878e-06,
         5.37533405e-06,  -2.78397019e-06,  -5.21942778e-06,
        -5.21942825e-06,   5.37533445e-06,  -5.21942748e-06,
        -5.21942778e-06,   4.22455732e-02,   4.22473086e-02,
        -5.21942748e-06,  -5.21942878e-06,  -5.21942778e-06,
         5.21942778e-06,   5.37533456e-06,   5.21220933e-06,
        -5.21943212e-06,  -2.78396471e-06,   5.21303725e-06,
         5.21229121e-06,   5.37532905e-06,  -2.40932524e-06,
         4.22455732e-02,  -5.21942778e-06,   4.22455696e-02,
         5.21942778e-06,   5.37532905e-06,  -5.21943155e-06,
         4.22455696e-02,   4.22455696e-02,  -5.21942778e-06,
         5.21229106e-06,   5.21942778e-06,   5.21942778e-06,
         5.21942778e-06,  -5.21942878e-06,   5.21942780e-06,
        -5.21942748e-06,   5.21942778e-06,   5.21942780e-06,
        -5.21942878e-06,   5.21229121e-06,  -5.21942878e-06,
         4.22455696e-02,   5.21942778e-06,   5.21942778e-06,
         5.21229209e-06,  -5.21942778e-06,   4.22455696e-02,
         5.21942778e-06,   5.37532905e-06,  -5.21942748e-06,
        -5.21942878e-06,   5.21229112e-06,   5.21942778e-06,
        -5.21942778e-06,   5.21229106e-06,   4.22455696e-02,
         5.21942778e-06,  -5.21942778e-06,   5.21942778e-06,
        -5.21943210e-06,  -5.21942878e-06,   4.22455696e-02,
        -5.21936489e-06,   5.21942778e-06,  -5.21942778e-06,
         5.21229121e-06,  -5.21942778e-06,   5.21942780e-06,
        -5.21942878e-06,   4.22455696e-02,  -5.21942778e-06,
        -5.21942878e-06,   5.21942778e-06,   4.22455696e-02,
         5.21942778e-06,   5.37532905e-06,   5.21942778e-06,
         5.37533445e-06,  -5.21942778e-06,  -5.21942778e-06,
         5.21942778e-06,   5.37532905e-06,   5.37532905e-06,
         5.21942778e-06,   5.21942780e-06,   5.21942780e-06,
         4.22455696e-02,   5.21942778e-06,   5.37532905e-06,
        -5.21936392e-06,   5.21942778e-06,   5.21942778e-06,
         4.22473088e-02,  -3.91020096e+00])

In [320]:
wFinal

array([  4.22455732e-02,   4.22473088e-02,   5.37630673e-06,
         4.22455732e-02,   5.37596623e-06,   4.22473088e-02,
         4.22455732e-02,   4.22473088e-02,   4.22473221e-02,
         4.22473222e-02,   4.22455732e-02,  -2.54292998e-06,
         4.22455696e-02,   4.22455732e-02,   5.37532906e-06,
         5.37634719e-06,  -5.21942825e-06,  -5.21937071e-06,
        -5.21942778e-06,   4.22455732e-02,   5.37634719e-06,
         5.37634675e-06,   5.37634719e-06,   4.22455696e-02,
         4.22473086e-02,   5.21942780e-06,   5.37533456e-06,
         4.22455732e-02,   5.37533405e-06,  -2.78179336e-06,
         4.22455732e-02,  -5.21942878e-06,   5.21229106e-06,
        -5.21943212e-06,  -2.78179336e-06,   4.22473222e-02,
         4.22455732e-02,   4.22455696e-02,   5.21229121e-06,
        -2.78179361e-06,   4.22455732e-02,  -2.40925210e-06,
        -5.21942778e-06,   5.37630675e-06,  -5.21942825e-06,
         4.22455696e-02,   4.22455696e-02,  -2.78179294e-06,
         4.22455696e-02,

In [7]:
def log_lik_normal(weights, x_i, r, sigma2):
    # we calculat the log likeligood given 
    return norm.logpdf(r, np.dot(weights[:-1], np.transpose(x_i)), sigma2 * np.identity(len(x_i)))

def normal_posterior(weights, x_i, br1, br2):
    # Same parameters as log_lik.
    # Note that we add up all the results to be able to batch process the final posterior probability
    return np.sum(log_lik_normal(weights, x_i, r, sigma2) + log_prior(weights[:-1]))

posteriorGradient = grad(normal_posterior, argnum=0)